# Wisconsin Visualization (2012)

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import PRGn, RdYlGn, Set3
from bokeh.transform import linear_cmap,factor_cmap
from bokeh.layouts import row, column
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
import numpy as np
import pandas as pd

In [ ]:
import pandas as pd

data = pd.read_csv("2012WIData.csv")
data.drop("Unnamed: 0",axis = 1, inplace=True)

data.head()

# Convert Latitude/Longitude to Mercator Coordinates

In [ ]:
# function below converts longitude/latitude to mercator coordinates because bokeh does not use longitude/latitude

# returns latitude as float (will get an error when showing visual if not a float)

def x_coord(x, y):
    
    lat = x
    lon = y
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return float(x)

data['Coordinates'] = list(zip(data['Residence_Addresses_Latitude'], data['Residence_Addresses_Longitude']))

mercator_x = [x_coord(x, y) for x, y in data['Coordinates'] ]
data["mercator_x"] = mercator_x

In [ ]:
# returns longitude as float (will get an error when showing visual if not a float)

def y_coord(x, y):
    
    lat = x
    lon = y
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return float(y)

data['Coordinates'] = list(zip(data['Residence_Addresses_Latitude'], data['Residence_Addresses_Longitude']))

mercator_y = [y_coord(x, y) for x, y in data['Coordinates'] ]
data["mercator_y"] = mercator_y

# New Column - Voted_in_2012 with Float Values

1) Replaced NaN values with 0 (plot does not show if NaN values present)

2) New column Voted_in_2012 where 0.0 = Did not Vote and 1.0 = Voted

In [ ]:
data["General_2012_11_06"] = data["General_2012_11_06"].fillna(0) #fill NaN values
data["Voted_in_2012"] = data["General_2012_11_06"].replace({0:0.0, "Y":1.0}) #1.0 = Voted in 2012 and 0.0 = did not vote
data.head()

In [ ]:
data = data.dropna()

In [ ]:
data.dtypes #Checking to make sure mercator columns and Voted_in_2012 column are floats

# Visualization

In [ ]:
chosentile = get_provider(Vendors.STAMEN_TONER)

In [ ]:
palette = Set3[10]
source = ColumnDataSource(data = data)

color_mapper = linear_cmap(field_name = 'Voted_in_2012', palette = palette, low = 0.0, high = 1.0)

In [ ]:
# Create figure
p = figure(title = '2012 Wisconsin Voter Location and Data', x_axis_type="mercator", y_axis_type="mercator", x_axis_label = 'Longitude', y_axis_label = 'Latitude')
p.add_tile(chosentile)


p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=4, fill_alpha = 0.7)

output_notebook()
output_file('Wisconsin_2012.html', title='2012 Wisconsin Voter Location and Data')

In [ ]:
show(p)